### 📘 L1-ranking: fast candidate cut (DuckDB)


In [1]:
import os
import duckdb as ddb

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
DATA_DIR = "../data/raw"  # где лежат parquet с "core" или полными фичами/текстами
TRAIN_PARQUET = os.path.join(
    DATA_DIR, "train-dset-001.parquet"
)  # или твой файл с текстами
TEST_PARQUET = os.path.join(DATA_DIR, "test-dset-small.parquet")

OUT_DIR = "../data/transformed"
os.makedirs(OUT_DIR, exist_ok=True)
N_Q_TRAIN = 200_000  # число запросов для train
K = 500  # целевой cut per query (подберём ниже по Recall@K)
USE_STEMMER = "russian"  # если тексты русские; можно 'none'

In [4]:
# Инициализация DuckDB
TMP_DIR = "./duck_tmp"
os.makedirs(TMP_DIR, exist_ok=True)

con = ddb.connect()
con.execute("PRAGMA threads=2;")  # меньше потоков — меньше пиковая RAM
con.execute("PRAGMA memory_limit='6GB';")  # подбери под свою машину (например, 6–8 GB)
con.execute(f"PRAGMA temp_directory='{os.path.abspath(TMP_DIR)}';")  # спиллы на диск
# опционально: прогресс
con.execute("PRAGMA enable_progress_bar=true;")

In [5]:
con.execute(
    f"CREATE OR REPLACE VIEW train_raw AS SELECT * FROM read_parquet('{TRAIN_PARQUET}');"
)
con.execute(
    f"CREATE OR REPLACE VIEW test_raw  AS SELECT * FROM read_parquet('{TEST_PARQUET}');"
)

con.execute("""
    INSTALL fts;
    LOAD fts;
""")

con.execute(
    f"CREATE OR REPLACE VIEW train_raw AS SELECT * FROM read_parquet('{TRAIN_PARQUET}');"
)
con.execute(
    f"CREATE OR REPLACE VIEW test_raw  AS SELECT * FROM read_parquet('{TEST_PARQUET}');"
)

print("Train size:", con.execute("SELECT COUNT(*) FROM train_raw;").fetchone()[0])
print("Test size: ", con.execute("SELECT COUNT(*) FROM test_raw;").fetchone()[0])
print(
    "Train queries:",
    con.execute("SELECT COUNT(DISTINCT query_id) FROM train_raw;").fetchone()[0],
)
print(
    "Test queries: ",
    con.execute("SELECT COUNT(DISTINCT query_id) FROM test_raw;").fetchone()[0],
)

Train size: 7781790
Test size:  335348


Train queries: 678190
Test queries:  12505


In [6]:
# случайные 500k уникальных query_id для train
con.execute(f"""
    CREATE OR REPLACE TEMP TABLE sample_q_train AS
    SELECT query_id
    FROM (
      SELECT DISTINCT query_id, random() AS r
      FROM train_raw
    )
    ORDER BY r
    LIMIT {N_Q_TRAIN}
""")

n_q_train = con.execute("SELECT COUNT(*) FROM sample_q_train;").fetchone()[0]
print("Уникальных train query_id:", n_q_train)

# случайные 500k уникальных query_id для test
con.execute("""
    CREATE OR REPLACE TEMP TABLE sample_q_test AS
    SELECT query_id
    FROM (
      SELECT DISTINCT query_id, random() AS r
      FROM test_raw
    )
    ORDER BY r
""")

n_q_test = con.execute("SELECT COUNT(*) FROM sample_q_test;").fetchone()[0]
print("Уникальных test query_id:", n_q_test)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Уникальных train query_id: 200000
Уникальных test query_id: 335348


In [7]:
# TRAIN: только выбранные 500k query_id
con.execute("""
    CREATE OR REPLACE TABLE train_fts AS
    SELECT
        row_number() OVER () AS rid,
        r.query_id, r.item_id,
        r.query_text,
        COALESCE(r.item_title, '') AS item_title,
        COALESCE(r.item_description, '') AS item_description,
        COALESCE(r.item_contact, 0) AS item_contact
    FROM train_raw r
    JOIN sample_q_train q USING (query_id)
""")

# TEST: только выбранные 500k query_id
con.execute("""
    CREATE OR REPLACE TABLE test_fts AS
    SELECT
        row_number() OVER () AS rid,
        r.query_id, r.item_id,
        r.query_text,
        COALESCE(r.item_title, '') AS item_title,
        COALESCE(r.item_description, '') AS item_description
    FROM test_raw r
    JOIN sample_q_test q USING (query_id)
""")

print("train_fts rows:", con.execute("SELECT COUNT(*) FROM train_fts;").fetchone()[0])
print("test_fts rows :", con.execute("SELECT COUNT(*) FROM test_fts;").fetchone()[0])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/traitlets/traitlets.py(708): __set__
  /tmp/ipykernel_91571/668717133.py(2): <module>
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3579): run_code
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3519): run_ast_nodes
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3336): run_cell_async
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/async_helpers.py(128): _pseudo_sync_runner
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3132): _run_cell
  /home/syrenny/Desktop/avito-for-applicant/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3077): run_cell
 

In [ ]:
# подгружаем FTS (если уже подгружено — просто продолжит)
try:
    con.execute("PRAGMA load_extension='fts';")
except Exception as e:
    print("FTS load note:", e)

# перестраховка: удалим прежние индексы, если были
for tbl in ("train_fts", "test_fts"):
    try:
        con.execute(f"PRAGMA drop_fts_index('{tbl}');")
    except Exception:
        pass

# создаём ОДИН индекс на title и (опционально) desc
con.execute(f"""
    PRAGMA create_fts_index(
        'train_fts', 'rid',
        'item_title', 'item_description',
        stemmer='{USE_STEMMER}', stopwords='russian', lower=1, overwrite=1
    );
""")

con.execute(f"""
    PRAGMA create_fts_index(
        'test_fts', 'rid',
        'item_title', 'item_description',
        stemmer='{USE_STEMMER}', stopwords='russian', lower=1, overwrite=1
    );
""")

print("FTS index created (train/test).")


In [ ]:
# в некоторых версиях нужен неймспейс fts_main_<table>; начнём с общего match_bm25
def compute_bm25_title(table_name: str):
    try:
        con.execute(f"""
            CREATE OR REPLACE TABLE {table_name}_bm25 AS
            SELECT
                t.*,
                match_bm25('{table_name}', rid, query_text, fields:='item_title') AS bm25_title
            FROM {table_name} t
        """)
    except Exception as e1:
        print(f"[{table_name}] generic match_bm25 failed, trying fts_main_*:", e1)
        con.execute(f"""
            CREATE OR REPLACE TABLE {table_name}_bm25 AS
            SELECT
                t.*,
                fts_main_{table_name}.match_bm25(rid, query_text, fields:='item_title') AS bm25_title
            FROM {table_name} t
        """)


compute_bm25_title("train_fts")
compute_bm25_title("test_fts")

# z-нормализация BM25 по запросу
for name in ("train_fts_bm25", "test_fts_bm25"):
    con.execute(f"""
        CREATE OR REPLACE TABLE {name} AS
        SELECT
            *,
            CAST( (bm25_title - AVG(bm25_title) OVER (PARTITION BY query_id))
                  / NULLIF(STDDEV_SAMP(bm25_title) OVER (PARTITION BY query_id), 0)
                  AS FLOAT
            ) AS bm25_title_z
        FROM {name}
    """)


In [ ]:
# Top-K по z-скорy BM25 (title)
con.execute(f"""
    CREATE OR REPLACE TABLE train_l1_topk AS
    WITH ranked AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY query_id ORDER BY bm25_title_z DESC NULLS LAST) AS rk
        FROM train_fts_bm25
    )
    SELECT *
    FROM ranked
    WHERE rk <= {K} OR item_contact = 1
""")

con.execute(f"""
    CREATE OR REPLACE TABLE test_l1_topk AS
    WITH ranked AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY query_id ORDER BY bm25_title_z DESC NULLS LAST) AS rk
        FROM test_fts_bm25
    )
    SELECT *
    FROM ranked
    WHERE rk <= {K}
""")

print(
    "train_l1_topk rows:",
    con.execute("SELECT COUNT(*) FROM train_l1_topk;").fetchone()[0],
)
print(
    "test_l1_topk rows :",
    con.execute("SELECT COUNT(*) FROM test_l1_topk;").fetchone()[0],
)


In [ ]:
# Recall@K: средняя доля позитивов, попавших в Top-K, среди запросов с >=1 позитивом
def recall_at_k(con) -> float:
    df = con.execute("""
        WITH pos AS (
            SELECT query_id, SUM(item_contact) AS pos_total
            FROM train_fts_bm25
            GROUP BY 1
        ),
        pos_topk AS (
            SELECT query_id, SUM(item_contact) AS pos_in_topk
            FROM train_l1_topk
            GROUP BY 1
        )
        SELECT p.query_id, p.pos_total, COALESCE(t.pos_in_topk,0) AS pos_in_topk
        FROM pos p LEFT JOIN pos_topk t USING(query_id)
    """).df()
    df = df[df.pos_total > 0]
    return float((df.pos_in_topk / df.pos_total).mean()) if len(df) else float("nan")


recall_k = recall_at_k(con)
print(f"Recall@{K} (train, on {n_q_train} queries): {recall_k:.4f}")


In [ ]:
train_out = os.path.join(OUT_DIR, f"train_l1_topk_{n_q_train}q.parquet")
test_out = os.path.join(OUT_DIR, f"test_l1_topk.parquet")

con.execute(f"""
COPY (
  SELECT
    query_id, item_id,
    item_contact,
    query_text, item_title, item_description,
    bm25_title, bm25_title_z
  FROM train_l1_topk
) TO '{train_out}' (FORMAT 'parquet');
""")

con.execute(f"""
COPY (
  SELECT
    query_id, item_id,
    query_text, item_title, item_description,
    bm25_title, bm25_title_z
  FROM test_l1_topk
) TO '{test_out}' (FORMAT 'parquet');
""")

print("Saved:", train_out)
print("Saved:", test_out)
print(f"Recall@{K}: {recall_k:.4f}")
